<a href="https://colab.research.google.com/github/rokmr/digitalVideo/blob/main/opticalFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import os
import cv2
import glob
import numpy as np
import imageio.v2 as imageio
import matplotlib.image
from pyoptflow import HornSchunck
from matplotlib import pyplot as plt
from typing import List
from scipy.ndimage import morphology
from tqdm import tqdm
import torch
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity

In [5]:
class IntermediateFrame():
  def __init__(self, I0, I1, t, u, v):
    self.I0 = I0
    self.I0_gray = np.dot(I0,[0.2989,0.5870,0.1140])
    self.I1 = I1
    self.I1_gray = np.dot(I1,[0.2989,0.5870,0.1140])
    self.t = t
    self.output = np.zeros_like(self.I0)
    self.m , self.n, self.o  = I0.shape
    self.y, self.x = np.float32(np.meshgrid(np.arange(self.n),np.arange(self.m)))

    # Claculation of forward optical flow
    self.u, self.v = u, v

    # Claculation of intermediate optical flow for any time t
    xt, yt = np.round(np.float32(self.x+t*self.u)).astype("uint32"),np.round(np.float32(self.y+t*self.v)).astype("uint32")

    self.Ut_x = np.zeros_like(self.I0_gray, dtype = np.float32)
    self.Ut_y = np.zeros_like(self.I0_gray, dtype = np.float32)

    minIntensity = np.zeros_like(self.I0_gray, dtype = np.float32)
    min_y = np.zeros_like(self.I0_gray, dtype = np.float32)
    mapped = np.zeros_like(self.I0_gray)

    for i in range(self.m):
      for j in range(self.n):

        k, l = xt[i,j], yt[i,j]
        if 0 <= k and k < self.m and 0 <= l and l < self.n:
          mapped[k,l] = 1
          diff = abs(self.I0_gray[i,j] - self.I1_gray[k,l])

          if not self.Ut_x[k,l] or not self.Ut_y[k,l]:
            self.Ut_x[k,l] = self.u[i,j]
            self.Ut_y[k,l] = self.v[i,j]

          else:
            if diff < minIntensity[k,l]:
              self.Ut_x[k,l] = self.u[i,j]
              self.Ut_y[k,l] = self.v[i,j]
              minIntensty[k,l] = diff

    # Outside In Strategy
    nonMapped = np.where(mapped==0)
    for i, j in zip(*nonMapped):
        if 0 < i and i < self.m-1 and 0 < j and j < self.n-1:
          neighbourhoodx = self.Ut_x[i - 1:i + 2, j - 1:j + 2]
          max_valuex = np.max(neighbourhoodx)
          self.Ut_x[i, j] = max_valuex

          neighbourhoody = self.Ut_y[i - 1:i + 2, j - 1:j + 2]
          max_valuey = np.max(neighbourhoody)
          self.Ut_y[i, j] = max_valuey

    # Occlusion Mask
    self.O0 = np.zeros_like(self.I0_gray).astype("uint8")
    self.O1 = np.ones_like(self.I0_gray).astype("uint8")
    xt1, yt1 = np.round(np.float32(self.x+self.u)).astype("uint32"),np.round(np.float32(self.y+self.v)).astype("uint32")
    for i in range(self.m):
      for j in range(self.n):
        k,l = xt1[i,j], yt1[i,j]
        if 0 <= k and k < self.m and 0 <= l and l < self.n:
          if np.sqrt((self.u[i,j] - self.Ut_x[k,l])**2 + (self.v[i,j] - self.Ut_y[k,l])**2) >0.5:
            self.O0[i,j]=1
          self.O1[k][l] = 0

    # Dilating Occlusion Mask
    kernel = np.ones((3,3),np.uint8)
    self.dilation_O0 = cv2.dilate(self.O0,kernel,iterations = 1)
    self.dilation_O1 = cv2.dilate(self.O1,kernel,iterations = 1)

    # Reconstruction of Frame
    X0_x, X0_y = np.uint(self.x - self.t * self.Ut_x) , np.uint(self.y - self.t * self.Ut_x)
    X1_x, X1_y = np.uint(self.x + (1 - self.t) *self.t * self.Ut_y), np.uint(self.y + (1 - self.t) *self.t * self.Ut_y)

    for i in range(self.m):
      for j in range(self.n):
        x0,y0=X0_x[i,j],X0_y[i,j]
        x1,y1=X1_x[i,j],X1_y[i,j]
        if self.dilation_O1[x1,y1]==1:
          self.output[i,j] = self.I0[x0,y0]
        elif self.dilation_O1[x1,y1]==0 and self.dilation_O0[x0,y0]==1:
          self.output[i,j] = self.I1[x1,y1]
        else:
          self.output[i,j] = (1-self.t)*self.I0[x0,y0] + (self.t)*self.I1[x1,y1]

# Multi-Scale Optical Flow

In [27]:
class multiScaleDiscrete_HSOF():
  def __init__(self, Img0, Img1, numScale:int):
    self.I0list = [Img0[::2**i, ::2**i] for i in range(numScale)]
    self.I1list = [Img1[::2**i, ::2**i] for i in range(numScale)]
    self.numScale = numScale
    self.U, self.V = HornSchunck(np.dot(self.I0list[-1],[0.2989,0.5870,0.1140]), np.dot(self.I1list[-1],[0.2989,0.5870,0.1140]), alpha=1, Niter=100)
    self.u, self.v = 0, 0

    # def process(self):
    for i in reversed(range(self.numScale-1)):
      U_u = cv2.resize(2*self.U, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
      U_v = cv2.resize(2*self.V, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)

      wr = self.image_warping(np.dot(self.I0list[i],[0.2989,0.5870,0.1140]), U_u, U_v)
      self.u, self.v= HornSchunck(np.dot(self.I1list[i],[0.2989,0.5870,0.1140]), wr, alpha=1, Niter=100)
      self.U, self.V = U_u + self.u , U_v + self.v

  def image_warping(self,frame0,a,b):
    x,y=np.float32(np.meshgrid(np.arange(frame0.shape[1]),np.arange(frame0.shape[0])))
    x1,y1=np.float32(x+a),np.float32(y+b)
    warped_image=cv2.remap(frame0,x1,y1,interpolation=cv2.INTER_CUBIC)
    return warped_image

# Reconstruction of Video

In [36]:
class repeatingFrame():
  def __init__(self, frameFolder:str, frameType:str, K:List, desFolder:str):
    self.frameFolder = frameFolder
    self.K = K
    self.framesPath = frameFolder+ "/*." + frameType
    self.pathList = glob.glob(self.framesPath)
    self.pathList.sort()
    self.frameList = [cv2.imread(framePath) for framePath in self.pathList]
    self.l = len(self.frameList)
    self.Task = "notComplete"

    if not os.path.exists(desFolder+f"/videos"):
      os.makedirs(desFolder+f"/videos")

    for k in self.K:
      if not os.path.exists(desFolder+f"/repeatedframe/{k}"):
            os.makedirs(desFolder+f"/repeatedframe/{k}")

      self.newFrameList = []
      for j, frame in enumerate(self.frameList):

        if j % k == 0:
          self.newFrameList.extend([frame] * k)

      self.Shape = self.newFrameList[0].shape
      fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
      video_writer = cv2.VideoWriter(f"{desFolder}/videos/output_repeatingFrame_{k}.mp4", fourcc, 15, (self.Shape[1], self.Shape[0]))

      for idx, frame in enumerate(self.newFrameList[:self.l]):
        cv2.imwrite(desFolder+f"/repeatedframe/{k}/{idx}.png", frame*255)
        video_writer.write(frame.astype("uint8"))

      video_writer.release()

    self.Task = "Completed"

In [37]:
RV1 = repeatingFrame("./00001/rgb","png", [1,2,3,4],"./00001")

In [38]:
RV6 = repeatingFrame("./00006/rgb","png", [1,2,3,4],"./00006")

In [8]:
def multiInterpolated(frameFolder, desFolder:str, K:List, s:int):
  pathList = glob.glob(frameFolder)
  pathList.sort()
  frameList = [cv2.imread(framePath) for framePath in pathList]

  for k in tqdm(K):
    if not os.path.exists(desFolder+f"/{s+1}scale/{k}"):
        os.makedirs(desFolder+f"/{s+1}scale/{k}")

    tempFrameList = []
    for frame in frameList[::k]:
          tempFrameList.append(frame)


    newFrameList = []
    for i in range(len(tempFrameList)-1):
      newFrameList.append(tempFrameList[i])
      for j in range(1,k):
        M = multiScaleDiscrete_HSOF(tempFrameList[i],tempFrameList[i+1],s+1)
        M.process()
        U, V = M.U, M.V
        IF = IntermediateFrame(tempFrameList[i],tempFrameList[i+1],(j+1)/k,U, V)
        newFrameList.append(IF.output)

    newFrameList.append(tempFrameList[-1])

    Shape = newFrameList[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    video_writer = cv2.VideoWriter(f"{desFolder}/videos/scale{s+1}output_multiInterpolated_{k}.mp4", fourcc, 15, (Shape[1], Shape[0]))

    for idx, frame in enumerate(newFrameList):
      cv2.imwrite(desFolder+f"/{s+1}scale/{k}/{idx}.png", frame*255)
      video_writer.write(frame.astype())

    video_writer.release()


In [42]:
class multiInterpolated(IntermediateFrame):
  def __init__(self, frameFolder:str, frameType:str, desFolder:str, K:List, numscale:int):
    self.frameFolder = frameFolder
    self.K = K
    self.numscale = numscale
    self.framesPath = frameFolder+ "/*." + frameType
    self.pathList = glob.glob(self.framesPath)
    self.pathList.sort()
    self.frameList = [cv2.imread(framePath)/255 for framePath in self.pathList]
    self.Task = "notComplete"

    if not os.path.exists(desFolder+f"/videos"):
      os.makedirs(desFolder+f"/videos")

    for s in range(0, self.numscale):
      for k in tqdm(self.K):
        if not os.path.exists(desFolder+f"/{s+1}scale/{k}"):
              os.makedirs(desFolder+f"/{s+1}scale/{k}")

        self.tempFrameList = []
        for frame in self.frameList[::k]:
            self.tempFrameList.append(frame)

        self.newFrameList = []
        for i in range(len(self.tempFrameList)-1):
          self.newFrameList.append(self.tempFrameList[i])
          for j in range(1,k):
            self.M = multiScaleDiscrete_HSOF(self.tempFrameList[i],self.tempFrameList[i+1],s+1)
            self.U, self.V = self.M.U, self.M.V
            self.IF = IntermediateFrame(self.tempFrameList[i],self.tempFrameList[i+1],j/k,self.U, self.V)
            self.newFrameList.append(self.IF.output)

        self.newFrameList.append(self.tempFrameList[len(self.tempFrameList)-1])


        self.Shape = self.newFrameList[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
        video_writer = cv2.VideoWriter(f"{desFolder}/videos/scale{s+1}output_multiInterpolated_{k}.mp4", fourcc, 15, (self.Shape[1], self.Shape[0]))

        for idx, frame in enumerate(self.newFrameList):
          cv2.imwrite(desFolder+f"/{s+1}scale/{k}/{idx}.png", frame*255)
          video_writer.write(frame.astype("uint8"))

        video_writer.release()

    self.Task = "Completed"


In [43]:
MM1 = multiInterpolated("./00001/rgb","png","./00001", [2,3,4], 2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [24:22<00:00, 487.36s/it]


In [ ]:
MM1 = multiInterpolated("./00001/rgb","png","./00001", [2,3,4], 2)

In [46]:
MM6 = multiInterpolated("./00006/rgb","png","./00006", [2,3,4], 4)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [24:33<00:00, 491.00s/it]


# Numerical Performance

In [10]:
class PSNRCalculator:
    def __init__(self, folder_path1, folder_path2, k):
        self.folder_path1 = folder_path1
        self.folder_path2 = folder_path2
        self.k = k

    @staticmethod
    def calculate_psnr(image1, image2):
        mse = np.mean((image1 - image2) ** 2)
        if mse == 0:
            return float('inf')
        max_pixel = 255.0
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr

    def remove_every_kth_item(self, my_list, k):
      index = 0
      while index < len(my_list):
          del my_list[index]
          index += k - 1

    def calculate_average_psnr(self):
        psnr_values = []
        file_list1 = sorted(os.listdir(self.folder_path1))
        file_list2 = sorted(os.listdir(self.folder_path2))
        self.remove_every_kth_item(file_list1, self.k)
        self.remove_every_kth_item(file_list2, self.k)

        for file1, file2 in zip(file_list1, file_list2):
            image_path1 = os.path.join(self.folder_path1, file1)
            image_path2 = os.path.join(self.folder_path2, file2)

            if os.path.isfile(image_path1) and os.path.isfile(image_path2):
                img1 = cv2.imread(image_path1)
                img2 = cv2.imread(image_path2)

                if img1.shape == img2.shape:
                    psnr = self.calculate_psnr(img1, img2)
                    psnr_values.append(psnr)

        if not psnr_values:
            return 0.0

        average_psnr = sum(psnr_values) / len(psnr_values)
        return average_psnr

In [11]:
# Example usage:
K = [2, 3, 4]
PSNR_repeatedframe = []
folder_path1 = "/home/rohitk/DVPA/A1/00001/rgb"
folder_path2 = "/home/rohitk/DVPA/A1/00001/1scale/"
for k in K:
  psnr_calculator = PSNRCalculator(folder_path1, folder_path2+f"{k}", k)
  average_psnr = psnr_calculator.calculate_average_psnr()
  PSNR_repeatedframe.append(f"{average_psnr:.4f}")
print(f"Average PSNR: {PSNR_repeatedframe=}")


Average PSNR: PSNR_repeatedframe=['27.8967', '27.8949', '27.8948']


In [12]:
refFolder_path1 = "/home/rohitk/DVPA/A1/00001/rgb"
refFolder_path6 = "/home/rohitk/DVPA/A1/00006/rgb"

def PSNR(calFolder_path:str, refFolder_path:str ):
  K = [2, 3, 4]
  PSNR_values = []
  for k in K:
    psnr_calculator = PSNRCalculator(refFolder_path, calFolder_path+f"{k}", k)
    average_psnr = psnr_calculator.calculate_average_psnr()
    PSNR_values.append(f"{average_psnr:.3f}")
  return PSNR_values

In [47]:
# Repeated Frame
PSNR_repeatedframe1 = PSNR("/home/rohitk/DVPA/A1/00001/repeatedframe/", refFolder_path1)
PSNR_repeatedframe6 = PSNR("/home/rohitk/DVPA/A1/00006/repeatedframe/", refFolder_path6)

# Scale 1: original
PSNR_s1_1 =  PSNR("/home/rohitk/DVPA/A1/00001/1scale/", refFolder_path1)
PSNR_s1_6 =  PSNR("/home/rohitk/DVPA/A1/00006/1scale/", refFolder_path6)

# Scale 2: original + down scale by 2 once
PSNR_s2_1 =  PSNR("/home/rohitk/DVPA/A1/00001/2scale/", refFolder_path1)
PSNR_s2_6 =  PSNR("/home/rohitk/DVPA/A1/00006/2scale/", refFolder_path6)

# Scale 3: original + down scale by 2 once + down scale by 2 twice
PSNR_s3_1 =  PSNR("/home/rohitk/DVPA/A1/00001/3scale/", refFolder_path1)
PSNR_s3_6 =  PSNR("/home/rohitk/DVPA/A1/00006/3scale/", refFolder_path6)

# Scale 4: original + down scale by 2 once + down scale by 2 twice + down scale by 2 thrice
PSNR_s4_1 =  PSNR("/home/rohitk/DVPA/A1/00001/4scale/", refFolder_path1)
PSNR_s4_6 =  PSNR("/home/rohitk/DVPA/A1/00006/4scale/", refFolder_path6)


In [73]:
import pandas as pd
data = {
    'Checked': ["Repeated frame", "Scale: 1",  "Scale: 2",  "Scale: 3", "Scale: 4",],
    'K=2': [str(PSNR_repeatedframe1[0]), str(PSNR_s1_1[0]), str(PSNR_s2_1[0]), str(PSNR_s3_1[0]),  str(PSNR_s4_1[0])],
    'K=3': [str(PSNR_repeatedframe1[1]), str(PSNR_s1_1[1]), str(PSNR_s2_1[1]), str(PSNR_s3_1[1]),  str(PSNR_s4_1[1])],
    'K=4': [str(PSNR_repeatedframe1[2]), str(PSNR_s1_1[2]), str(PSNR_s2_1[2]), str(PSNR_s3_1[2]),  str(PSNR_s4_1[2])]
}

df = pd.DataFrame(data)
print("PSNR output for ImageFolder: 00001")
print(df)

PSNR output for ImageFolder: 00001
          Checked     K=2     K=3     K=4
0  Repeated frame  27.839  27.838  27.849
1        Scale: 1  31.484  31.281  31.303
2        Scale: 2  31.498  31.280  31.299
3        Scale: 3  31.509  31.286  31.300
4        Scale: 4  31.511  31.292  31.308


In [49]:
import pandas as pd
data = {
    'Checked': ["Repeated frame", "Scale: 1",  "Scale: 2",  "Scale: 3", "Scale: 4",],
    'K=2': [str(PSNR_repeatedframe6[0]), str(PSNR_s1_6[0]), str(PSNR_s2_6[0]), str(PSNR_s3_6[0]), str(PSNR_s4_6[0])],
    'K=3': [str(PSNR_repeatedframe6[1]), str(PSNR_s1_6[1]), str(PSNR_s2_6[1]), str(PSNR_s3_6[1]), str(PSNR_s4_6[1])],
    'K=4': [str(PSNR_repeatedframe6[2]), str(PSNR_s1_6[2]), str(PSNR_s2_6[2]), str(PSNR_s3_6[2]), str(PSNR_s4_6[2])]
}

df = pd.DataFrame(data)

print("PSNR output for ImageFolder: 00006")
print(df)



PSNR output for ImageFolder: 00006
          Checked     K=2     K=3     K=4
0  Repeated frame  27.975  27.972  27.975
1        Scale: 1  29.629  29.518  29.525
2        Scale: 2  29.624  29.511  29.517
3        Scale: 3  29.620  29.509  29.518
4        Scale: 4  29.614  29.511  29.524


In [64]:
class LPIPSCalculator:
    def __init__(self, folder_path1, folder_path2, k):
        self.folder_path1 = folder_path1
        self.folder_path2 = folder_path2
        self.k = k
        self.transform = transforms.ToTensor()
        self.lpips = LearnedPerceptualImagePatchSimilarity(net_type='vgg')



    def calculate_average_lpips(self):
        lpips_values = []
        file_list1 = sorted(os.listdir(self.folder_path1))
        file_list2 = sorted(os.listdir(self.folder_path2))


        for file1, file2 in zip(file_list1, file_list2):
            image_path1 = os.path.join(self.folder_path1, file1)
            image_path2 = os.path.join(self.folder_path2, file2)

            if os.path.isfile(image_path1) and os.path.isfile(image_path2):
                img1 = Image.open(image_path1)
                img2 = Image.open(image_path2)

                lpips = self.lpips(self.transform(img1).unsqueeze(0), self.transform(img2).unsqueeze(0))

                lpips_values.append(lpips)

        if not lpips_values:
            return 0.0

        average_psnr = sum(lpips_values) / len(lpips_values)
        return average_psnr

In [65]:
refFolder_path1 = "/home/rohitk/DVPA/A1/00001/rgb"
refFolder_path6 = "/home/rohitk/DVPA/A1/00006/rgb"

def LPIPS(calFolder_path:str, refFolder_path:str ):
  K = [2, 3, 4]
  LPIPS_values = []
  for k in K:
    lpips_calculator = LPIPSCalculator(refFolder_path, calFolder_path+f"{k}", k)
    average_lpips = lpips_calculator.calculate_average_lpips()
    LPIPS_values.append(f"{average_lpips:.3f}")
  return LPIPS_values

In [61]:
import torchvision.transforms as transforms
from PIL import Image

In [66]:
# Repeated Frame
LPIPS_repeatedframe1 = LPIPS("/home/rohitk/DVPA/A1/00001/repeatedframe/", refFolder_path1)
LPIPS_repeatedframe6 = LPIPS("/home/rohitk/DVPA/A1/00006/repeatedframe/", refFolder_path6)

# Scale 1: original
LPIPS_s1_1 =  LPIPS("/home/rohitk/DVPA/A1/00001/1scale/", refFolder_path1)
LPIPS_s1_6 =  LPIPS("/home/rohitk/DVPA/A1/00006/1scale/", refFolder_path6)

# Scale 2: original + down scale by 2 once
LPIPS_s2_1 =  LPIPS("/home/rohitk/DVPA/A1/00001/2scale/", refFolder_path1)
LPIPS_s2_6 =  LPIPS("/home/rohitk/DVPA/A1/00006/2scale/", refFolder_path6)

# Scale 3: original + down scale by 2 once + down scale by 2 twice
LPIPS_s3_1 =  LPIPS("/home/rohitk/DVPA/A1/00001/3scale/", refFolder_path1)
LPIPS_s3_6 =  LPIPS("/home/rohitk/DVPA/A1/00006/3scale/", refFolder_path6)

# Scale 4: original + down scale by 2 once + down scale by 2 twice + down scale by 2 thrice
LPIPS_s4_1 =  LPIPS("/home/rohitk/DVPA/A1/00001/4scale/", refFolder_path1)
LPIPS_s4_6 =  LPIPS("/home/rohitk/DVPA/A1/00006/4scale/", refFolder_path6)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/rohitk/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 528M/528M [00:06<00:00, 87.4MB/s]


In [69]:
import pandas as pd
data = {
    'Checked': ["Repeated frame", "Scale: 1",  "Scale: 2",  "Scale: 3", "Scale: 4",],
    'K=2': [str(LPIPS_repeatedframe1[0]), str(LPIPS_s1_1[0]), str(LPIPS_s2_1[0]), str(LPIPS_s3_1[0]),  str(LPIPS_s4_1[0])],
    'K=3': [str(LPIPS_repeatedframe1[1]), str(LPIPS_s1_1[1]), str(LPIPS_s2_1[1]), str(LPIPS_s3_1[1]),  str(LPIPS_s4_1[1])],
    'K=4': [str(LPIPS_repeatedframe1[2]), str(LPIPS_s1_1[2]), str(LPIPS_s2_1[2]), str(LPIPS_s3_1[2]),  str(LPIPS_s4_1[2])]
}

df = pd.DataFrame(data)
print("LPIPS output for ImageFolder: 00001")
print(df)

LPIPS output for ImageFolder: 00001
          Checked    K=2    K=3    K=4
0  Repeated frame  0.615  0.612  0.608
1        Scale: 1  0.268  0.270  0.272
2        Scale: 2  0.267  0.269  0.270
3        Scale: 3  0.265  0.266  0.266
4        Scale: 4  0.265  0.267  0.267


In [72]:
import pandas as pd
data = {
    'Checked': ["Repeated frame", "Scale: 1",  "Scale: 2",  "Scale: 3", "Scale: 4",],
    'K=2': [str(LPIPS_repeatedframe6[0]), str(LPIPS_s1_6[0]), str(LPIPS_s2_6[0]), str(LPIPS_s3_6[0]), str(LPIPS_s4_6[0])],
    'K=3': [str(LPIPS_repeatedframe6[1]), str(LPIPS_s1_6[1]), str(LPIPS_s2_6[1]), str(LPIPS_s3_6[1]), str(LPIPS_s4_6[1])],
    'K=4': [str(LPIPS_repeatedframe6[2]), str(LPIPS_s1_6[2]), str(LPIPS_s2_6[2]), str(LPIPS_s3_6[2]), str(LPIPS_s4_6[2])]
}

df = pd.DataFrame(data)

print("LPIPS output for ImageFolder: 00006")
print(df)

LPIPS output for ImageFolder: 00006
          Checked    K=2    K=3    K=4
0  Repeated frame  0.568  0.563  0.558
1        Scale: 1  0.379  0.379  0.380
2        Scale: 2  0.380  0.380  0.381
3        Scale: 3  0.383  0.385  0.387
4        Scale: 4  0.385  0.390  0.394
